# Project 3: Smart Beta Portfolio and Portfolio Optimization

## Overview


Smart beta has a broad meaning, but we can say in practice that when we use the universe of stocks from an index, and then apply some weighting scheme other than market cap weighting, it can be considered a type of smart beta fund.  A Smart Beta portfolio generally gives investors exposure or "beta" to one or more types of market characteristics (or factors) that are believed to predict prices while giving investors a diversified broad exposure to a particular market. Smart Beta portfolios generally target momentum, earnings quality, low volatility, and dividends or some combination. Smart Beta Portfolios are generally rebalanced infrequently and follow relatively simple rules or algorithms that are passively managed.  Model changes to these types of funds are also rare requiring prospectus filings with US Security and Exchange Commission in the case of US focused mutual funds or ETFs.. Smart Beta portfolios are generally long-only, they do not short stocks.

In contrast, a purely alpha-focused quantitative fund may use multiple models or algorithms to create a portfolio. The portfolio manager retains discretion in upgrading or changing the types of models and how often to rebalance the portfolio in attempt to maximize performance in comparison to a stock benchmark.  Managers may have discretion to short stocks in portfolios.

Imagine you're a portfolio manager, and wish to try out some different portfolio weighting methods.

One way to design portfolio is to look at certain accounting measures (fundamentals) that, based on past trends, indicate stocks that produce better results.  


For instance, you may start with a hypothesis that dividend-issuing stocks tend to perform better than stocks that do not. This may not always be true of all companies; for instance, Apple does not issue dividends, but has had good historical performance.  The hypothesis about dividend-paying stocks may go something like this: 

Companies that regularly issue dividends may also be more prudent in allocating their available cash, and may indicate that they are more conscious of prioritizing shareholder interests.  For example, a CEO may decide to reinvest cash into pet projects that produce low returns.  Or, the CEO may do some analysis, identify that reinvesting within the company produces lower returns compared to a diversified portfolio, and so decide that shareholders would be better served if they were given the cash (in the form of dividends).  So according to this hypothesis, dividends may be both a proxy for how the company is doing (in terms of earnings and cash flow), but also a signal that the company acts in the best interest of its shareholders.  Of course, it's important to test whether this works in practice.


You may also have another hypothesis, with which you wish to design a portfolio that can then be made into an ETF.  You may find that investors may wish to invest in passive beta funds, but wish to have less risk exposure (less volatility) in their investments.  The goal of having a low volatility fund that still produces returns similar to an index may be appealing to investors who have a shorter investment time horizon, and so are more risk averse.

So the objective of your proposed portfolio is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  Also, if this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

Smart Beta ETFs can be designed with both of these two general methods (among others): alternative weighting and minimum volatility ETF.


## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 880kB 15.8MB/s ta 0:00:01    56% |██████████████████▎             | 501kB 31.0MB/s eta 0:00:01
    100% |████████████████████████████████| 12.2MB 3.0MB/s eta 0:00:01  3% |█                               | 378kB 24.8MB/s eta 0:00:01    99% |███████████████████████████████▉| 12.1MB 27.9MB/s eta 0:00:01
    100% |████████████████████████████████| 26.2MB 1.5MB/s eta 0:00:01   23% |███████▍                        | 6.1MB 29.9MB/s eta 0:00:01    38% |████████████▍                   | 10.2MB 29.8MB/s eta 0:00:01    43% |██████████████                  | 11.5MB 26.4MB/s eta 0:00:01    48% |███████████████▋                | 12.8MB 25.4MB/s eta 0:00:01    69% |██████████████████████▏         | 18.2MB 26.1MB/s eta 0:00:01
    100% |████████████████████████████████| 1.1MB 17.4MB/s ta 0:00:01
    100% |████████████████████████████████| 50.0MB 860kB/s eta 0:00:01  0% |▏                               | 194kB 27.6MB/s eta 0:00:02    5% |█▊                   

### Load Packages

In [2]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [3]:
%ls -l

total 140
drwxr-xr-x 3 root root  4096 Feb 28 03:23 graphs/
-rw-r--r-- 1 root root   606 Sep 11  2018 helper.py
-rw-r--r-- 1 root root 57184 Mar  1 02:34 project_3_starter.ipynb
-rw-r--r-- 1 root root 34846 Sep 18  2019 project_3_starter-zh.ipynb
-rw-r--r-- 1 root root  7956 Sep 11  2018 project_helper.py
-rw-r--r-- 1 root root  9204 Sep 11  2018 project_tests.py
drwxr-xr-x 2 root root  4096 Sep 18  2019 __pycache__/
-rw-r--r-- 1 root root   214 Nov 30  2018 requirements.txt
-rw-r--r-- 1 root root  6506 Sep 11  2018 tests.py


In [4]:
df = pd.read_csv('../../data/project_3/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [5]:
df.head(5)

,date,ticker,adj_close,adj_volume,dividends
1009,2013-07-01,AAL,16.17609308,12511796.00000000,0.00000000
1010,2013-07-02,AAL,15.81983388,10748794.00000000,0.00000000
1011,2013-07-03,AAL,16.12794994,7039678.00000000,0.00000000
1012,2013-07-05,AAL,16.21460758,6426810.00000000,0.00000000
1013,2013-07-08,AAL,16.31089385,7161394.00000000,0.00000000


In [6]:
df.tail()

,date,ticker,adj_close,adj_volume,dividends
483669,2017-06-26,XOM,78.12543603,7718724.00000000,0.00000000
483670,2017-06-27,XOM,78.00041995,9221642.00000000,0.00000000
483671,2017-06-28,XOM,78.40431807,9189671.00000000,0.00000000
483672,2017-06-29,XOM,77.60613845,20525077.00000000,0.00000000
483673,2017-06-30,XOM,77.63498832,13996452.00000000,0.00000000


In [7]:
df.shape

(99705, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99705 entries, 1009 to 483673
Data columns (total 5 columns):
date          99705 non-null object
ticker        99705 non-null object
adj_close     99705 non-null float64
adj_volume    99705 non-null float64
dividends     99705 non-null float64
dtypes: float64(3), object(2)
memory usage: 4.6+ MB


In [9]:
df.describe()

,adj_close,adj_volume,dividends
count,99705.00000000,99705.00000000,99705.00000000
mean,98.02639661,10991865.10524046,0.01335722
std,120.56949483,15680370.84530087,1.83564616
min,3.72955891,7900.00000000,0.00000000
25%,41.02678343,3519014.00000000,0.00000000
50%,67.19143629,6262961.00000000,0.00000000
75%,99.31889312,12163900.00000000,0.00000000
max,1011.34000000,616620500.00000000,567.97166850


In [10]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
In Part 1 of this project, you'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF.  You'll compare this portfolio to a market cap weighted index to see how well it performs. 

Note that in practice, you'll probably get the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's), but for this exercise we will simulate a market cap weighted index.

## Index Weights
The index we'll be using is based on large dollar volume stocks. Implement `generate_dollar_volume_weights` to generate the weights for this index. For each date, generate the weights based on dollar volume traded for that date. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [11]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    
    print(f"\nStart:dollar_volume")
    dollar_volume = close * volume
    print(f"{dollar_volume}")
    print(f"{type(dollar_volume)}")
    print(f"End:dollar_volume\n")
    
    print(f"\nStart:dollar_volume_sum")
    dollar_volume_sum = dollar_volume.sum(axis=1) # sum along columns
    print(f"{dollar_volume_sum}")
    print(f"{type(dollar_volume_sum)}")
    print(f"End:dollar_volume_sum\n")
    
    print(f"\nStart:dollar_volume_weights")
    dollar_volume_weights = dollar_volume.divide(dollar_volume_sum, axis=0) # divide along rows
    print(f"{dollar_volume_weights}")
    print(f"{type(dollar_volume_weights)}")
    print(f"End:dollar_volume_weights\n")

    return dollar_volume_weights


project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)


Start:dollar_volume
                           HJF                 IWG                QSYL
2005-02-01  348628075.71300000  608648315.28000009  300410784.98600000
2005-02-02 7575630049.36999989 6240087760.44999981 4380507847.72000027
2005-02-03  941144343.83999991  754203279.77999997  553581860.38199997
2005-02-04  364742751.09999996 5254224178.90000057  545086266.30700004
<class 'pandas.core.frame.DataFrame'>
End:dollar_volume


Start:dollar_volume_sum
2005-02-01    1257687175.97900009
2005-02-02   18196225657.54000092
2005-02-03    2248929484.00199986
2005-02-04    6164053196.30700111
dtype: float64
<class 'pandas.core.series.Series'>
End:dollar_volume_sum


Start:dollar_volume_weights
                  HJF        IWG       QSYL
2005-02-01 0.27719777 0.48394253 0.23885970
2005-02-02 0.41632975 0.34293308 0.24073717
2005-02-03 0.41848548 0.33536102 0.24615350
2005-02-04 0.05917255 0.85239760 0.08842984
<class 'pandas.core.frame.DataFrame'>
End:dollar_volume_weights

Tests Passed


### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [12]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')


Start:dollar_volume
ticker                    AAL                 AAPL               ABBV  \
date                                                                    
2013-07-01 202391976.65295082  5192133342.44938660 160240491.76812646   
2013-07-02 170044135.53290918  6379858749.09848881 154675436.02818424   
2013-07-03 113535574.39577270  3289403473.24362564  85687954.74287605   
2013-07-05 104208202.16722879  3711212490.37529945  98378473.62004057   
2013-07-08 116808737.36893333  4014865794.27665520 145851994.43971246   
2013-07-09 182685199.45438606  4831570151.80062866 138374228.00378600   
2013-07-10 190130912.43986583  3841394533.31340551 113516885.36293092   
2013-07-11 141413052.51851743  4523559972.82930374 150603022.78256321   
2013-07-12 178919709.25073317  3868672078.74982738 217263513.19007871   
2013-07-15 149777857.10430652  3355016082.93688917 123424483.27640234   
2013-07-16 177889120.79958883  3022401649.64293003 137833816.92883056   
2013-07-17 299658647.97497630 

## Portfolio Weights
Now that we have the index weights, let's choose the portfolio weights based on dividend. You would normally calculate the weights based on trailing dividend yield, but we'll simplify this by just calculating the total dividend yield over time.

Implement `calculate_dividend_weights` to return the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.
For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [13]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    
    #TODO: Implement function
    print(f"\nStart:dividends")
    print(f"{dividends}")
    print(f"{type(dividends)}")
    print(f"End:dividends\n")
    
    print(f"\nStart:cumulative sum of dividends")
    cum_sum_of_dividends = dividends.cumsum()
    print(f"{cum_sum_of_dividends}")
    print(f"{type(cum_sum_of_dividends)}")
    print(f"End:cumulative sum of dividends\n")
    
    
    print(f"\nStart:sum of cumulative sum of dividends")
    sum_of_cum_sum_of_dividends  = cum_sum_of_dividends.sum(axis=1)
    print(f"{sum_of_cum_sum_of_dividends}")
    print(f"{type(sum_of_cum_sum_of_dividends)}")
    print(f"End:sum of cumulative sum of dividendss\n")
    
    
    print(f"\nStart:dividend_weights")
    dividend_weights  = cum_sum_of_dividends.divide(sum_of_cum_sum_of_dividends, axis=0)
    print(f"{dividend_weights}")
    print(f"{type(dividend_weights)}")
    print(f"End:dividend_weights\n")
    
    return dividend_weights




project_tests.test_calculate_dividend_weights(calculate_dividend_weights)


Start:dividends
                 BJXP        JOP        VEO
2009-07-13 0.00000000 0.00000000 0.00000000
2009-07-14 0.00000000 0.00000000 0.10000000
2009-07-15 0.00000000 1.00000000 0.30000000
2009-07-16 0.00000000 0.20000000 0.00000000
<class 'pandas.core.frame.DataFrame'>
End:dividends


Start:cumulative sum of dividends
                 BJXP        JOP        VEO
2009-07-13 0.00000000 0.00000000 0.00000000
2009-07-14 0.00000000 0.00000000 0.10000000
2009-07-15 0.00000000 1.00000000 0.40000000
2009-07-16 0.00000000 1.20000000 0.40000000
<class 'pandas.core.frame.DataFrame'>
End:cumulative sum of dividends


Start:sum of cumulative sum of dividends
2009-07-13   0.00000000
2009-07-14   0.10000000
2009-07-15   1.40000000
2009-07-16   1.60000000
dtype: float64
<class 'pandas.core.series.Series'>
End:sum of cumulative sum of dividendss


Start:dividend_weights
                 BJXP        JOP        VEO
2009-07-13        nan        nan        nan
2009-07-14 0.00000000 0.00000000 1.0000000

### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [14]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')


Start:dividends
ticker            AAL       AAPL       ABBV        ABT        AGN        AIG  \
date                                                                           
2013-07-01 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-02 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-03 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-05 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-08 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-09 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-10 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-11 0.00000000 0.00000000 0.40000000 0.14000000 0.00000000 0.00000000   
2013-07-12 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-15 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-16 0.00000000 0

## Returns
Implement `generate_returns` to generate returns data for all the stocks and dates from price data. You might notice we're implementing returns and not log returns. Since we're not dealing with volatility, we don't have to use log returns.

In [15]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    
    #TODO: Implement function
    print(f"\nPrices")
    print(f"{prices}")
    
    print(f"\nPrices shift by 1")
    print(f"{prices.shift(1)}")
    
    returns = (prices - prices.shift(1)).div(prices.shift(1))
    print(f"\nReturns")
    print(f"{returns}")

    return returns


project_tests.test_generate_returns(generate_returns)


Prices
                  WALM         DFE        PXRY
2012-10-27 35.44110000 34.17990000 34.02230000
2012-10-28 92.11310000 91.05430000 90.95720000
2012-10-29 57.97080000 57.78140000 58.19820000
2012-10-30 34.17050000 92.45300000 58.51070000

Prices shift by 1
                  WALM         DFE        PXRY
2012-10-27         nan         nan         nan
2012-10-28 35.44110000 34.17990000 34.02230000
2012-10-29 92.11310000 91.05430000 90.95720000
2012-10-30 57.97080000 57.78140000 58.19820000

Returns
                  WALM         DFE        PXRY
2012-10-27         nan         nan         nan
2012-10-28  1.59904743  1.66397210  1.67345829
2012-10-29 -0.37065629 -0.36541822 -0.36015840
2012-10-30 -0.41055669  0.60004777  0.00536958
Tests Passed


### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [16]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')


Prices
ticker             AAL         AAPL        ABBV         ABT          AGN  \
date                                                                       
2013-07-01 16.17609308  53.10917319 34.92447839 31.42538772 122.62751990   
2013-07-02 15.81983388  54.31224742 35.42807578 31.27288084 121.05361758   
2013-07-03 16.12794994  54.61204262 35.44486235 30.72565028 121.21003024   
2013-07-05 16.21460758  54.17338125 35.85613355 31.32670680 123.53666845   
2013-07-08 16.31089385  53.86579916 36.66188936 31.76628544 123.65397794   
2013-07-09 16.71529618  54.81320389 36.35973093 31.16522893 122.89146626   
2013-07-10 16.53235227  54.60295791 36.85493502 31.16522893 121.28823656   
2013-07-11 16.72492481  55.45406479 37.08155384 31.85599537 122.19738511   
2013-07-12 16.90786872  55.35309481 38.15724076 31.81096287 123.35092842   
2013-07-15 17.10044125  55.47379158 37.79303181 31.95506689 122.37334934   
2013-07-16 17.28338516  55.83133953 37.10696377 32.15320992 120.08581429   
2013

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [17]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function
    
    weighted_returns = returns * weights

    return weighted_returns

project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [18]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [19]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function
    
    print(f"\nStart: Returns")
    print(f"{returns}")
    print(f"Stop: Returns\n")
    
    print(f"\nStart: Returns sum")
    returns_sum = returns.sum(axis=1) + 1
    print(f"{returns_sum}")
    print(f"Stop: Returns\n")
    
    print(f"\nStart: cumulative_returns")
    cumulative_returns = returns_sum.cumprod()
    print(f"{cumulative_returns}")
    print(f"{type(cumulative_returns)}")
    print(f"\nStop: cumulative_returns")
    
    return cumulative_returns

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)


Start: Returns
                   BVE         LFV         LRT
2010-08-06         nan         nan         nan
2010-08-07  1.59904743  1.66397210  1.67345829
2010-08-08 -0.37065629 -0.36541822 -0.36015840
2010-08-09 -0.41055669  0.60004777  0.00536958
Stop: Returns


Start: Returns sum
2010-08-06           nan
2010-08-07    5.93647782
2010-08-08   -0.09623291
2010-08-09    1.19486066
dtype: float64
Stop: Returns


Start: cumulative_returns
2010-08-06           nan
2010-08-07    5.93647782
2010-08-08   -0.57128454
2010-08-09   -0.68260542
dtype: float64
<class 'pandas.core.series.Series'>

Stop: cumulative_returns
Tests Passed


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [20]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')


Start: Returns
ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00008007  0.00308984  0.00004768 -0.00001880 -0.00003423   
2013-07-03  0.00009093  0.00074662  0.00000167 -0.00018529  0.00000450   
2013-07-05  0.00001716 -0.00091335  0.00003497  0.00008758  0.00005552   
2013-07-08  0.00001590 -0.00052257  0.00007514  0.00007710  0.00000238   
2013-07-09  0.00009451  0.00177325 -0.00002380 -0.00012663 -0.00001420   
2013-07-10 -0.00004895 -0.00034661  0.00003637  0.00000000 -0.00004299   
2013-07-11  0.00003254  0.00139306  0.00001830  0.00012582  0.00002055   
2013-07-12  0.00003548 -0.00012769  0.00011425 -0.00000507  0.00002572   
2013-07-15  0.00003910  0.00016769 -0.00002700  0.00002153 -0.00002676   
2013-07-16  0.00004287  0.00043884 -0.00005637  0.00003814 -0.00005587   
2013-07-17  0.00018322

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [21]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    
    tracking_error = np.sqrt(252) * (etf_returns_by_date - benchmark_returns_by_date).std()

    return tracking_error

project_tests.test_tracking_error(tracking_error)

Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [22]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.1020761483200753


# Part 2: Portfolio Optimization

Now, let's create a second portfolio.  We'll still reuse the market cap weighted index, but this will be independent of the dividend-weighted portfolio that we created in part 1.

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [23]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
    
    returns_covariance = np.cov(returns.fillna(0).transpose())
    print(f"{returns_covariance}")
    
    return returns_covariance

project_tests.test_get_covariance_returns(get_covariance_returns)

[[0.89856076 0.7205586  0.8458721 ]
 [0.7205586  0.78707297 0.76450378]
 [0.8458721  0.76450378 0.83182775]]
Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [24]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

[[5.31791451e-04 7.21251856e-05 9.55847963e-05 ... 4.55526842e-05
  1.26756746e-04 4.27885833e-05]
 [7.21251856e-05 2.14185487e-04 4.87000321e-05 ... 3.43023358e-05
  8.17766960e-05 4.13786801e-05]
 [9.55847963e-05 4.87000321e-05 2.64537496e-04 ... 3.32734532e-05
  1.11616723e-04 5.40669112e-05]
 ...
 [4.55526842e-05 3.43023358e-05 3.32734532e-05 ... 1.14424083e-04
  3.48568802e-05 2.79122492e-05]
 [1.26756746e-04 8.17766960e-05 1.11616723e-04 ... 3.48568802e-05
  7.17162013e-04 8.42633668e-05]
 [4.27885833e-05 4.13786801e-05 5.40669112e-05 ... 2.79122492e-05
  8.42633668e-05 1.29561995e-04]]


### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [25]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    #TODO: Implement function
    print(f"\nStart: index_weights length")
    index_weights_length = index_weights.shape[0]
    print(f"{index_weights}")
    print(f"{index_weights_length}")
    print(f"Stop: index_weights lengths\n")
    
    print(f"\nStart: x = optimization_variable")
    x = cvx.Variable(index_weights_length)
    print(f"{x}n")
    print(f"End: x = optimization_variable\n")
    
    
    print(f"\nStart: covariance_returns in Quadratic form")
    covariance_returns_quadratic_form = cvx.quad_form(x, covariance_returns)
    print(f"{covariance_returns_quadratic_form}n")
    print(f"End:covariance_returns in Quadratic form\n")
    
    
    print(f"\nStart: Euclidean distance (L2 norm) between Portfolio Covariance returns and index weights")
    L2_norm = cvx.norm( x - index_weights, 2) # for L2-norm using argument 2
    print(f"{L2_norm}n")
    print(f"End:Euclidean distance (L2 norm) between Portfolio Covariance returns and index weights\n")
    
    
    print(f"\nStart: Objective function")
    objective_function = cvx.Minimize(covariance_returns_quadratic_form + ( L2_norm *  scale))
    print(f"{objective_function}n")
    print(f"End: Objective function\n")
    
    
    print(f"\nStart: Constraints")
    constraints = [x >= 0, sum(x) == 1]
    print(f"{constraints}n")
    print(f"End: Constraints\n")
    
    print(f"\nStart: Problem solution, Optimal Value of x")
    problem_solution =  cvx.Problem(objective_function, constraints ).solve()
    #print(f"{problem_solution}n")
    print(f"{x.value}n")
    print(f"End:  Problem solution\n")
    
    return x.value
    
    
project_tests.test_get_optimal_weights(get_optimal_weights)


Start: index_weights length
A   0.23623892
B   0.01256280
C   0.75119820
dtype: float64
3
Stop: index_weights lengths


Start: x = optimization_variable
var0n
End: x = optimization_variable


Start: covariance_returns in Quadratic form
QuadForm(var0, [[0.143123   0.0216755  0.014273  ]
 [0.0216755  0.0401826  0.00663152]
 [0.014273   0.00663152 0.044963  ]])n
End:covariance_returns in Quadratic form


Start: Euclidean distance (L2 norm) between Portfolio Covariance returns and index weights
Pnorm(var0 + -[0.23623892 0.0125628  0.7511982 ], 2)n
End:Euclidean distance (L2 norm) between Portfolio Covariance returns and index weights


Start: Objective function
minimize QuadForm(var0, [[0.143123   0.0216755  0.014273  ]
 [0.0216755  0.0401826  0.00663152]
 [0.014273   0.00663152 0.044963  ]]) + Pnorm(var0 + -[0.23623892 0.0125628  0.7511982 ], 2) * 2.0n
End: Objective function


Start: Constraints
[NonPos(Expression(AFFINE, UNKNOWN, (3,))), Zero(Expression(AFFINE, UNKNOWN, ()))]n
End: Con

## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [26]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)


Start: index_weights length
ticker
AAL     0.00505383
AAPL    0.04996914
ABBV    0.00507628
ABT     0.00344529
AGN     0.00631443
AIG     0.00723481
AMAT    0.00628914
AMGN    0.00690689
AMZN    0.05022972
APC     0.00367206
AVGO    0.00923578
AXP     0.00420389
BA      0.00659224
BAC     0.03065369
BIIB    0.00570220
BMY     0.00386121
C       0.01591180
CAT     0.00755899
CBS     0.00230404
CELG    0.00784448
CHTR    0.00719021
CMCSA   0.01067014
CMG     0.00471339
COP     0.00600604
COST    0.01150549
CRM     0.00522686
CSCO    0.01085341
CVS     0.00530558
CVX     0.00845056
DAL     0.00402521
           ...    
NVDA    0.04032500
ORCL    0.00993157
OXY     0.00412432
PEP     0.00568072
PFE     0.00669019
PG      0.00783857
PM      0.00553949
PXD     0.00366138
QCOM    0.00714256
REGN    0.00952813
SBUX    0.00709645
SLB     0.00780563
T       0.01234318
TGT     0.00308681
TWX     0.00712188
TXN     0.00539759
UAL     0.00398087
UNH     0.00669447
UNP     0.00567573
UPS     0.0035

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [27]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))


Start: Returns
ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00011130  0.00113194  0.00007320 -0.00001672 -0.00008104   
2013-07-03  0.00009843  0.00027582  0.00000241 -0.00006029  0.00000816   
2013-07-05  0.00002715 -0.00040137  0.00005890  0.00006740  0.00012121   
2013-07-08  0.00003001 -0.00028371  0.00011407  0.00004834  0.00000600   
2013-07-09  0.00012530  0.00087887 -0.00004184 -0.00006519 -0.00003894   
2013-07-10 -0.00005531 -0.00019167  0.00006914  0.00000000 -0.00008238   
2013-07-11  0.00005887  0.00077888  0.00003121  0.00007636  0.00004733   
2013-07-12  0.00005528 -0.00009098  0.00014726 -0.00000487  0.00005961   
2013-07-15  0.00005756  0.00010896 -0.00004845  0.00001561 -0.00005004   
2013-07-16  0.00005407  0.00032207 -0.00009215  0.00002136 -0.00011804   
2013-07-17  0.00014078

Optimized ETF Tracking Error: 0.05795012630412267


## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [28]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    all_rebalance_weights = []
    for shift in range(chunk_size, len(returns), shift_size):
        start_date_index = shift - chunk_size
        covariance_returns = get_covariance_returns(returns.iloc[start_date_index:shift])
        
        all_rebalance_weights.append(get_optimal_weights(covariance_returns, index_weights.iloc[shift-1]))
    
    
    
    return all_rebalance_weights

project_tests.test_rebalance_portfolio(rebalance_portfolio)

[[ 2.97086366e-04 -1.49593881e-04 -8.82401092e-05]
 [-1.49593881e-04  1.68909932e-04  3.42159759e-05]
 [-8.82401092e-05  3.42159759e-05  5.57551203e-05]]

Start: index_weights length
ZSD    0.00395679
SRG    0.12434660
WLBK   0.00335064
Name: 2005-06-14, dtype: float64
3
Stop: index_weights lengths


Start: x = optimization_variable
var68n
End: x = optimization_variable


Start: covariance_returns in Quadratic form
QuadForm(var68, [[ 2.97086366e-04 -1.49593881e-04 -8.82401092e-05]
 [-1.49593881e-04  1.68909932e-04  3.42159759e-05]
 [-8.82401092e-05  3.42159759e-05  5.57551203e-05]])n
End:covariance_returns in Quadratic form


Start: Euclidean distance (L2 norm) between Portfolio Covariance returns and index weights
Pnorm(var68 + -[0.00395679 0.1243466  0.00335064], 2)n
End:Euclidean distance (L2 norm) between Portfolio Covariance returns and index weights


Start: Objective function
minimize QuadForm(var68, [[ 2.97086366e-04 -1.49593881e-04 -8.82401092e-05]
 [-1.49593881e-04  1.6890993

Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [29]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

[[ 4.97130575e-04 -1.24172500e-05  7.30794234e-05 ...  2.45480367e-05
   1.22731182e-04  1.94181846e-05]
 [-1.24172500e-05  2.19133647e-04  7.45003912e-06 ...  1.28703252e-05
   3.20681483e-05 -5.87829479e-06]
 [ 7.30794234e-05  7.45003912e-06  2.09188084e-04 ...  3.19815432e-05
   6.61102206e-05  3.25454411e-05]
 ...
 [ 2.45480367e-05  1.28703252e-05  3.19815432e-05 ...  5.15389752e-05
   2.10346413e-05  1.98359580e-05]
 [ 1.22731182e-04  3.20681483e-05  6.61102206e-05 ...  2.10346413e-05
   3.61902991e-04  2.23353416e-05]
 [ 1.94181846e-05 -5.87829479e-06  3.25454411e-05 ...  1.98359580e-05
   2.23353416e-05  7.10258630e-05]]

Start: index_weights length
ticker
AAL     0.01043134
AAPL    0.06006002
ABBV    0.01145332
ABT     0.00460319
AGN     0.00899074
AIG     0.00659741
AMAT    0.00588434
AMGN    0.00543873
AMZN    0.01913355
APC     0.00440472
AVGO    0.00117287
AXP     0.00498903
BA      0.00982399
BAC     0.02170975
BIIB    0.00656725
BMY     0.00762632
C       0.01871608
CAT  

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover. We'll be using the formulas used in the classroom:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [31]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    print(f"\nStart: rebalanced_weights")
    rebalanced_weights = pd.DataFrame(all_rebalance_weights)
    print(f"{rebalanced_weights}n")
    print(f"Stop: rebalanced_weights\n")
    
    print(f"\nStart: sum_of_total_turnover")
    sum_of_total_turnover = (np.abs(rebalanced_weights - rebalanced_weights.shift(-1))).fillna(0).values.sum()
    print(f"{sum_of_total_turnover}")
    print(f"Stop: sum_of_total_turnover\n")
    
    print(f"\nStart: portfolio_turnover")
    portfolio_turnover = (sum_of_total_turnover * n_trading_days_in_year)/(rebalance_count * shift_size)
    print(f"{portfolio_turnover}")
    print(f"Stop: portfolio_turnover\n")
    
    
    return portfolio_turnover

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)


Start: rebalanced_weights
           0          1          2
0 0.00012205 0.00030199 0.99957596
1 0.00001306 0.00000811 0.99997883
2 0.39174818 0.56076878 0.04748304n
Stop: rebalanced_weights


Start: sum_of_total_turnover
1.905797323172624
Stop: sum_of_total_turnover


Start: portfolio_turnover
80.0434875732502
Stop: portfolio_turnover

Tests Passed


Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [32]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))


Start: rebalanced_weights
            0          1          2          3          4          5   \
0   0.01043134 0.06006002 0.01145332 0.00460319 0.00899074 0.00659741   
1   0.01064740 0.06444048 0.00623301 0.00369916 0.01256357 0.00614116   
2   0.00769333 0.06903315 0.01497718 0.00441796 0.00520380 0.00469487   
3   0.00616613 0.07389262 0.03434482 0.00630641 0.00493682 0.00425496   
4   0.00849226 0.07608856 0.01047717 0.00303387 0.00557975 0.00716587   
5   0.00546108 0.06935885 0.00851619 0.00243419 0.03065271 0.00725948   
6   0.01574075 0.07837675 0.01282181 0.00357958 0.00914547 0.00837615   
7   0.00686136 0.08294770 0.00882481 0.00359991 0.00969361 0.00546425   
8   0.00560596 0.10550724 0.01136890 0.00240813 0.00629966 0.00939865   
9   0.00639348 0.10950969 0.00933782 0.00436677 0.00481393 0.00590568   
10  0.00592953 0.09310054 0.00699162 0.00224169 0.00702000 0.00834156   
11  0.00570557 0.11661509 0.00877819 0.00241597 0.00741154 0.00589622   
12  0.00732166 0.0968674

That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.